[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/rag_attack/blob/main/rag_partie1.ipynb)

# RAG Agentique avec LangGraph - VéloCorp

Ce notebook démontre un système RAG agentique utilisant LangGraph pour orchestrer l'accès à 3 sources de données :
- 🔍 **Azure Search** - Documents, FAQ, manuels
- 🏢 **Base de données** - Données métier (commandes, clients, produits)
- 📊 **API CRM** - Commerciaux, prospects, opportunités

In [10]:
# Installations nécessaires

!pip install azure-core \
azure-identity \
azure-search\
azure-search-documents \
certifi\
python-dotenv \
faiss-cpu \
ipykernel \
langchain-community \
langchain-core \
langchain-huggingface \
langchain-openai \
langgraph \
pyodbc\
requests \
sentence-transformers

In [ ]:
!pip install \
    azure-identity \
    azure-mgmt-resource \
    azure-mgmt-search \
    azure-mgmt-sql \
    azure-mgmt-cognitiveservices \
    azure-mgmt-web \
    azure-keyvault-secrets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.1/124.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.9/912.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.7/653.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 8.3 MB/s eta 0:00:00


In [8]:
# Nécessaire d'effecture ceci avant d'importer Azure Search sinon Azure utilise
# sa nomenclature par défaut
import os
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "embedding"

In [ ]:
# Imports standards :
import functools
import json
import logging
import os
import re
import subprocess
import time
from datetime import datetime
from typing import Annotated, Dict, Optional, TypedDict

# Imports Azure
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

try:
    from azure.search.documents.models import VectorizedQuery
except ImportError:
    VectorizedQuery = None

from azure.identity import (
    DeviceCodeCredential,
    EnvironmentCredential,
    ChainedTokenCredential,
)
from azure.keyvault.secrets import SecretClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.search import SearchManagementClient
from azure.mgmt.sql import SqlManagementClient
from azure.mgmt.web import WebSiteManagementClient
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

# Imports third-party
import pyodbc
import requests
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import AzureSearch
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import (
    ChatOpenAI,
    AzureChatOpenAI,
    AzureOpenAIEmbeddings,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from typing import List, Optional, Dict, Any
from google.colab import files

In [14]:
# Ce bloc permet d’installer et de configurer le driver ODBC 18 de Microsoft pour se connecter à SQL Server.

# 1. Mise à jour de la liste des paquets et installation des outils ODBC
!apt-get update
!apt-get install -y unixodbc unixodbc-dev

# 2. Installation du driver MS ODBC 18
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!apt-get install -y curl gnupg
!curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list | tee /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql18

# 3. Vérification
!odbcinst -j
!odbcinst -q -d
!odbcinst -q -d -n "ODBC Driver 18 for SQL Server"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://cli.github.com/packages stable InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,876 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [

# 1/ RAG simple

Etape 1 : charger les manuels utilisateur des vélos

In [15]:
# Récupérer le fichier local
uploaded = files.upload()

# Récupérer le nom du fichier uploadé dans colab
filename = next(iter(uploaded))

Saving faq_manuels_mail (1).json to faq_manuels_mail (1).json


In [16]:
def manuals_json_loader(json_path: str) -> List[Dict]:
    """
    Charge uniquement les documents de type 'manuel_technique' à partir d'un fichier JSON unifié.

    Args:
        json_path (str): Chemin vers le fichier JSON fusionné contenant tous les documents.

    Returns:
        list of dict: Liste de dictionnaires contenant l'id, les métadonnées et le texte des manuels techniques.
    """
    # Ouvre le fichier JSON et charge toutes les données en mémoire
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    manuals = []  # Liste qui stockera tous les manuels techniques extraits

    # Parcourt chaque entrée du JSON unifié
    for key, item in data.items():
        meta = item.get("metadata", {})
        # On vérifie que le document est bien un manuel technique
        if meta and meta.get("type", "").lower() == "manuel_technique":
            manuals.append({
                "id": key,                # identifiant du document (ex: "manuel_e_sport")
                "metadata": meta,         # toutes les métadonnées associées
                "text": item.get("text", "")  # le texte intégral du manuel
            })
    return manuals

manuals = manuals_json_loader(filename)
manuals[0]

{'id': 'manuel_urbain_classic',
 'metadata': {'type': 'manuel_technique',
  'model': 'Urbain-Classic',
  'product_line': 'Urbain',
  'version': '2.5',
  'last_updated': '2025-03-13',
  'word_count': 368},
 'text': "Document ID: 20\nCreated: 2025-03-13\nFilename: manuel_urbain_classic.txt\n--------------------------------------------------\n\nManuel Technique - Urbain-Classic\n        \nBienvenue dans le manuel technique de votre Urbain-Classic. Ce document contient toutes les informations nécessaires pour l'utilisation, l'entretien et la maintenance de votre vélo.\n\nSPÉCIFICATIONS TECHNIQUES - Urbain-Classic\n\nPoids: 17.2 kg\nMatériau du cadre: acier\nTailles disponibles: S, M, L, XL\nCouleurs: noir, blanc, bleu\nGarantie: 12 mois\n\nComposants:\n- Freins: Freins à disque hydrauliques\n- Transmission: Dérailleur 21 vitesses\n- Roues: Jantes aluminium double paroi\n- Pneus: Pneus tout-terrain 700x35c\n- Selle: Selle ergonomique avec gel\n\nMONTAGE ET INSTALLATION\n\n1. Déballage\n   -

In [17]:
def split_by_section(text: str) -> List[str]:
    """
    Découpe un texte en sections à partir des titres de section en majuscules.
    Chaque chunk commence par un titre, suivi de son contenu.

    Args:
        text (str): Le texte du manuel complet.
    Returns:
        list of str: Liste des sections trouvées dans le texte.
    """
    # Regex pour trouver les titres de section (grandes majuscules sur une ligne)
    section_pattern = r"\n([A-ZÉÈÀÇ\- ]{8,})\n"
    # On split sur chaque titre détecté, ce qui donne :
    # [introduction, spécification techniques, montage, résolution de probèmes, ...]
    sections = re.split(section_pattern, text)
    chunks = []
    # sections[0] = introduction éventuelle (avant le premier titre)
    intro = sections[0].strip()
    if intro:
        chunks.append(intro)  # On garde l'intro si elle existe
    # On parcourt les couples (titre, contenu)
    for i in range(1, len(sections), 2): # pas de 2 pour sauter les contenus
        title = sections[i].strip()
        content = sections[i+1].strip() if i+1 < len(sections) else ""
        if content:
            # On recompose "titre + contenu" pour chaque chunk
            chunk = f"{title}\n{content}"
            chunks.append(chunk)
    return chunks

def make_chunk_with_context(doc: Dict[str, Any], section_chunk: str) -> str:
    """
    Ajoute un en-tête de contexte (modèle, type, ligne de produit) au début de chaque chunk.
    Cela permet à chaque chunk de garder l'identité du produit même si le texte ne le mentionne pas.

    Args:
        doc (dict): Dictionnaire contenant 'metadata' sur le produit.
        section_chunk (str): Le texte du chunk (section).
    Returns:
        str: Le chunk enrichi du contexte produit.
    """
    header = f"Modèle : {doc['metadata'].get('model', '')}\n"
    header += f"Type : {doc['metadata'].get('type', '')}\n"
    header += f"Ligne de produit : {doc['metadata'].get('product_line', '')}\n"
    return header + section_chunk


def chunking(
        manuals: List[Dict[str, Any]],
        chunk_size: int = 350,
        chunk_overlap: int = 50
    ) -> List[Dict[str, Any]]:
    """
    Découpe chaque manuel technique en chunks de taille fixe, avec overlap,
    en gardant le contexte du modèle dans chaque chunk.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    all_chunks = []
    for doc in manuals:
        # Nettoyage des séparateurs visuels
        clean_text = re.sub(r"(^[-\s]{3,}$\n?)+", "", doc["text"], flags=re.MULTILINE)
        # Découpage global du manuel en chunks overlapping
        chunks = splitter.split_text(clean_text)
        for i, chunk in enumerate(chunks):
            chunk_text = make_chunk_with_context(doc, chunk)
            all_chunks.append({
                "doc_id": doc["id"],
                "chunk_id": f"{doc['id']}_chunk_{i}",
                "metadata": doc["metadata"],
                "text": chunk_text
            })
    return all_chunks

Utilisation du code de chunking et visualisation des résultats :

In [18]:
all_chunks = chunking(manuals)

# Affichage d'exemple
for chunk in all_chunks[:2]:
    print("="*100)
    print(chunk["text"])

Modèle : Urbain-Classic
Type : manuel_technique
Ligne de produit : Urbain
Document ID: 20
Created: 2025-03-13
Filename: manuel_urbain_classic.txt
Manuel Technique - Urbain-Classic
Bienvenue dans le manuel technique de votre Urbain-Classic. Ce document contient toutes les informations nécessaires pour l'utilisation, l'entretien et la maintenance de votre vélo.

SPÉCIFICATIONS TECHNIQUES - Urbain-Classic
Modèle : Urbain-Classic
Type : manuel_technique
Ligne de produit : Urbain
SPÉCIFICATIONS TECHNIQUES - Urbain-Classic

Poids: 17.2 kg
Matériau du cadre: acier
Tailles disponibles: S, M, L, XL
Couleurs: noir, blanc, bleu
Garantie: 12 mois


### Désormais, nous allons vectoriser (embedding) nos chunks et les stocker pour créer un Vector Store.
Un Vector Store est une base de données spécialisée qui stocke des représentations numériques (embeddings) de textes ou documents pour permettre une recherche sémantique rapide.
Elle permet de retrouver efficacement les passages les plus pertinents par similarité de sens, même si les mots exacts ne sont pas présents dans la requête de l’utilisateur.

FAISS est une bibliothèque open source permettant la recherche ultra-rapide de similarité entre vecteurs dans de grands volumes de données.
On l’utilise pour retrouver efficacement les passages ou documents les plus pertinents à partir d’une requête, même dans des bases contenant des millions de textes ou d’images.
Parmi les alternatives, on trouve ChromaDB, Qdrant, Weaviate, Milvus, ou encore Annoy et ScaNN.


In [21]:
def create_faiss_vectorstore(
        embedded_chunks: List[Dict[str, Any]],
        model_name: str = 'sentence-transformers/all-MiniLM-L6-v2'
    ) -> FAISS:
    """
    Prend la liste de chunks embeddés, construit un vector store FAISS compatible LangChain/LangGraph.
    Retourne le vector store prêt à l'emploi.
    """
    # 1. Création des objets Document avec contenu et métadonnées
    docs = [
        Document(
            page_content=chunk["text"],
            metadata=chunk["metadata"]
        )
        for chunk in embedded_chunks
    ]

    # 2. Création de la fonction d'embedding (wrapper LangChain)
    embedding_fn = HuggingFaceEmbeddings(model_name=model_name)

    # 3. Création du vector store FAISS
    vectorstore = FAISS.from_documents(docs, embedding_fn)

    return vectorstore

In [22]:
# Utilisation de la fonction
vectorstore = create_faiss_vectorstore(all_chunks)
print(f"Vector store FAISS créé avec {vectorstore.index.ntotal} chunks.")

/tmp/ipython-input-3047090054.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_fn = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store FAISS créé avec 100 chunks.


In [23]:
# Stockage du vectorstore
# Chemin de sauvegarde :
persist_directory = "/content/data/faiss_vectorstore"
os.makedirs(persist_directory, exist_ok=True)
print(f"Dossier créé à : {persist_directory}")

# Sauvegarde du vector store
vectorstore.save_local(persist_directory)
print(f"Vector store FAISS sauvegardé à : {persist_directory}")

Dossier créé à : /content/data/faiss_vectorstore
Vector store FAISS sauvegardé à : /content/data/faiss_vectorstore


Nous pouvons dorénavant utiliser notre RAG :

In [24]:
# L'utilisateur saisit sa requête en langage naturel
query = "Quel est le poids du vélo Sport-Elite ?"

Inutile de générer manuellement l'embedding de la requête !
La méthode 'similarity_search' va automatiquement :
   - Passer la requête dans le même modèle d'embedding utilisé pour l'index
   - Comparer le vecteur résultant à tous les embeddings des chunks
   - Retourner les k passages les plus proches (selon la similarité cosinus)
C'est donc un appel "texte en entrée" ➔ "texte(s) le(s) plus pertinent(s) en sortie".


In [25]:
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n--- Résultat #{i+1} ---")
    print("Texte du chunk :")
    print(doc.page_content)
    print("Métadonnées :")
    print(doc.metadata)


--- Résultat #1 ---
Texte du chunk :
Modèle : Sport-Elite
Type : manuel_technique
Ligne de produit : Sport
SPÉCIFICATIONS TECHNIQUES - Sport-Elite

Poids: 8.5 kg
Matériau du cadre: carbone
Tailles disponibles: S, M, L, XL
Couleurs: gris, bleu
Garantie: 24 mois
Métadonnées :
{'type': 'manuel_technique', 'model': 'Sport-Elite', 'product_line': 'Sport', 'version': '1.5', 'last_updated': '2024-08-28', 'word_count': 367}

--- Résultat #2 ---
Texte du chunk :
Modèle : Sport-Elite
Type : manuel_technique
Ligne de produit : Sport
Problème: Bruit anormal
Solution: Identifiez la source, vérifiez le serrage des composants

Pour tout problème persistant, contactez notre service technique au 01 23 45 67 89.
Métadonnées :
{'type': 'manuel_technique', 'model': 'Sport-Elite', 'product_line': 'Sport', 'version': '1.5', 'last_updated': '2024-08-28', 'word_count': 367}

--- Résultat #3 ---
Texte du chunk :
Modèle : Sport-Elite
Type : manuel_technique
Ligne de produit : Sport
Entretien annuel:
- Révision

1/ Instanciation du LLM

In [ ]:
config = {

}

llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = "2024-02-15-preview"
)

2/ Exemple de prompt bien formulé pour que le LLM fasse de la synthèse RAG

In [31]:
prompt = """
Tu es un assistant expert, spécialisé dans la documentation technique des vélos.

Réponds de façon précise et concise à la question utilisateur suivante,
en t’appuyant STRICTEMENT et EXCLUSIVEMENT sur les extraits ci-dessous (issus de la documentation technique officielle) :
\"\"\"
{context}
\"\"\"

INSTRUCTIONS :
- Ne donne aucune information qui ne provient pas explicitement de ces extraits.
- Si la réponse exacte (par exemple, une valeur numérique, un modèle, un tableau) est présente, cite-la textuellement et indique d'où elle provient.
- Si plusieurs passages sont nécessaires pour une réponse complète, combine-les de façon claire.
- Si l’information demandée n’apparaît pas dans les extraits, dis simplement : « L’information ne figure pas dans la documentation fournie. »
- Reste factuel, sans extrapolation ni interprétation.

Question :
{query}
"""

3/ Récupérer les chunks les plus pertinents pour la requête

In [28]:
query = "Quel est le poids vélo Urbain-Classic ?"
results = vectorstore.similarity_search(query, k=2)  # on prend les 2 passages les plus pertinents
context = "\n\n".join([doc.page_content for doc in results])

4/ Construire le prompt à envoyer au LLM, en insérant le contexte récupéré et envoyer au LLM

In [32]:
response = llm.invoke(prompt.format(context=context, query=query))
print("Réponse du LLM :\n", response.content)

Réponse du LLM :
 Le poids du vélo Urbain-Classic est de 17.2 kg (source : SPÉCIFICATIONS TECHNIQUES - Urbain-Classic).


6/ comparaison avec une réponse générée sans RAG

In [33]:
print(llm.invoke(query).content)

Le poids du vélo Urbain-Classic peut varier en fonction de la marque et du modèle spécifiques. En général, les vélos urbains classiques pèsent entre 12 et 15 kilogrammes. Pour obtenir l'information la plus précise, il serait idéal de consulter les spécifications du fabricant ou du vendeur de ce modèle particulier.


7/ Test pour une question dont les éléments de réponse sont absents de la documentation

In [34]:
query = "De quel matériau est fabriqué le vélo urbain-confort ?"
results = vectorstore.similarity_search(query, k=2)
context = "\n\n".join([doc.page_content for doc in results])
response = llm.invoke(prompt.format(context=context, query=query))
print("Réponse du LLM :\n", response.content)

Réponse du LLM :
 L’information ne figure pas dans la documentation fournie.


## 2/ Construction et utilisation d'un RAG encapsulé dans une classe avec Azure
#### Ce RAG suit la même logique que précédemment, mais en utilisant Azure. Il est encapsulé dans une classe afin de pouvoir le réutiliser comme Tool pour notre RAG agentique.

#### Instanciation des LLMs via Azure

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = "2024-02-15-preview"
)


#### Création d'une classe pour récupérer uniquement les manuels utilisateurs

In [35]:
# -----------------------------------------------------------------------------
# AzureSearchFilteredRetriever
# -----------------------------------------------------------------------------
# Petit utilitaire pour interroger directement un index Azure Cognitive Search
# en mode "vector search" + filtre OData (ex: ne récupérer que les documents
# dont le champ 'type' vaut 'manuel').
#
# Pourquoi ce helper ?
# - Le wrapper LangChain AzureSearch ne propage pas toujours bien les filtres.
# - Ici on utilise le SDK officiel azure-search-documents pour garder le contrôle.
#
# Paramètres attendus :
#   endpoint   : URL du service Azure Search (ex: "https://xxx.search.windows.net")
#   index_name : nom de l'index (ex: "documents")
#   key        : clé admin ou query key autorisée
#   embed_fn   : fonction Python qui transforme un texte en vecteur (liste de floats)
#
# Méthode principale :
#   search_manuals(query, k=5, filter_expr="type eq 'manuel'")
#     -> renvoie les résultats bruts (SearchResult-like) retournés par Azure.
# -----------------------------------------------------------------------------

class AzureSearchFilteredRetriever:
    def __init__(self, endpoint: str, index_name: str, key: str, embed_fn):
        # Enregistre les paramètres de connexion / embedding
        self.endpoint = endpoint
        self.index_name = index_name
        self.embed_fn = embed_fn
        # Client natif Azure Search
        self.client = SearchClient(endpoint, index_name, AzureKeyCredential(key))

    def search_manuals(self, query: str, k: int = 5, filter_expr: str = "type eq 'manuel'"):
        # 1. embed la requête
        vec = self.embed_fn(query)
        # 2. construit l’appel selon SDK
        if VectorizedQuery is not None:
            # Nouveau SDK
            vq = VectorizedQuery(vector=vec, k=k, fields="embedding")
            results = self.client.search(
                search_text="*",  # pas de fulltext, vector only
                vector_queries=[vq],
                filter=filter_expr,
            )
        else:
            # Ancien SDK
            from azure.search.documents.models import Vector  # fallback
            v = Vector(value=vec, k=k, fields="embedding")
            results = self.client.search(
                search_text="*",
                vector=v,
                filter=filter_expr,
            )

        docs = []
        for r in results:
            # r est un dict-like
            docs.append(r)
        return docs


#### Création d'une classe pour le RAG

In [36]:
# -----------------------------------------------------------------------------
# Configuration logging simple pour afficher les étapes du pipeline.
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)


# -----------------------------------------------------------------------------
# RAGPipelineAzure
# -----------------------------------------------------------------------------
# Ce pipeline combine :
#   - un modèle d'embedding HuggingFace (utilisé pour vectoriser les requêtes)
#   - un LLM hébergé sur Azure OpenAI (chat completions)
#   - un index Azure Cognitive Search contenant des documents vectorisés
#
# self.search_client : client natif Azure + retrieve_filtered() -> filtre OData
#
# Usage typique :
#   pipeline = RAGPipelineAzure(...config Terraform...)
#   réponse = pipeline.answer("Ma question", k=5)
#
# NB : l'index contient un champ 'type' (filterable=True) et
#       un champ vectoriel 'embedding' (dimensions cohérentes avec l'embedding_fn).
# -----------------------------------------------------------------------------

class RAGPipelineAzure:
    def __init__(
        self,
        model_name: str,
        api_key: str,
        api_base: str,
        api_version: str,
        deployment_name: str,
        config: Dict[str, Any],
        index_name: str = "documents",
    ) -> None:

        """
        Initialise la pipeline RAG avec Azure Cognitive Search vectoriel.
        """
        # Sauvegarde du nom du modèle d'embedding HuggingFace
        self.model_name = model_name
        # Initialise la fonction d'embedding (requêtes & docs)
        self.embedding_fn = HuggingFaceEmbeddings(model_name=model_name)
        # Client LLM Azure OpenAI (déploiement chat)
        self.llm = AzureChatOpenAI(
            azure_endpoint=api_base,
            api_key=api_key,
            api_version=api_version,
            deployment_name=deployment_name,
        )
        # Nom de l'index Search
        self.index_name = index_name

        # Client natif Search pour filtrage propre
        self.search_client = SearchClient(
            config["search_endpoint"],
            index_name,
            AzureKeyCredential(config["search_key"])
        )

        # Petite lambda utilitaire : embed une requête texte -> vecteur (liste de floats)
        self._embed_query = lambda txt: self.embedding_fn.embed_query(txt)

    def retrieve_filtered(
        self,
        query: str,
        k: int = 5,
        doc_type: Optional[str] = None
    ) -> List[Document]:

        """Vector search côté Azure + filtre OData sur type='...'."""
        # Embed de la requête utilisateur
        vec = self._embed_query(query)
        # Expression de filtre OData (doit matcher un champ filterable dans l'index)
        filter_expr = f"type eq '{doc_type}'" if doc_type else None

        results_iter = None
        try:
            # Tentative nouvelle API
            from azure.search.documents.models import VectorizedQuery
            vq = VectorizedQuery(vector=vec, k=k, fields="embedding")
            results_iter = self.search_client.search(
                search_text="*",  # vector-only pattern
                vector_queries=[vq],
                filter=filter_expr,
            )
        except ImportError:
            # Ancienne API
            from azure.search.documents.models import Vector
            v = Vector(value=vec, k=k, fields="embedding")
            results_iter = self.search_client.search(
                search_text="*",
                vector=v,
                filter=filter_expr,
            )
        except Exception as e:
            # Toute autre erreur (réseau, auth, schéma, etc.)
            logger.error(f"Recherche Azure Search échouée: {e}")
            raise

        # Conversion vers objets LangChain Document pour réutiliser la suite du pipeline
        from langchain.schema import Document
        docs = []
        for r in results_iter:
            # r est SearchResult -> accès dict-like
            docs.append(
                Document(
                    page_content=r.get("content", "") if hasattr(r, "get") else r["content"],
                    metadata={
                        "id": r["id"] if "id" in r else getattr(r, "id", None),
                        "filename": r.get("filename") if hasattr(r, "get") else r.get("filename", None) if isinstance(r, dict) else None,
                        "type": r.get("type") if hasattr(r, "get") else r.get("type", None) if isinstance(r, dict) else None,
                        "created": r.get("created") if hasattr(r, "get") else r.get("created", None) if isinstance(r, dict) else None,
                        "document_id": r.get("document_id") if hasattr(r, "get") else r.get("document_id", None) if isinstance(r, dict) else None,
                    },
                )
            )
        # On ne retourne que les k premiers (Azure en a déjà retourné k, mais on sécurise)
        return docs[:k]

    def answer(self, query: str, k: int = 5) -> str:
        """Renvoie une réponse générée par le LLM sur la base des docs filtrés."""
        try:
            # Récupération des meilleurs chunks filtrés côté Azure (type='manuel')
            results = self.retrieve_filtered(query, k=k, doc_type="manuel")
            # Concaténation du contenu des chunks pour former le contexte
            context = "\n\n".join([doc.page_content for doc in results])
            # Prompt instructif (contexte + question)
            prompt = f"""
                Tu es un assistant expert, spécialisé dans la documentation technique des vélos.

                Réponds de façon précise et concise à la question utilisateur suivante,
                en t’appuyant STRICTEMENT et EXCLUSIVEMENT sur les extraits ci-dessous (issus de la documentation technique officielle) :
                \"\"\"
                {context}
                \"\"\"

                INSTRUCTIONS :
                - Ne donne aucune information qui ne provient pas explicitement de ces extraits.
                - Si la réponse exacte (par exemple, une valeur numérique, un modèle, un tableau) est présente, cite-la textuellement et indique d'où elle provient.
                - Si plusieurs passages sont nécessaires pour une réponse complète, combine-les de façon claire.
                - Si l’information demandée n’apparaît pas dans les extraits, dis simplement : « L’information ne figure pas dans la documentation fournie. »
                - Reste factuel, sans extrapolation ni interprétation.

                Question :
                {query}
                """
            # Appel LLM Azure OpenAI
            logger.info("Envoi du prompt au LLM Azure OpenAI...")
            response = self.llm.invoke(prompt)
            logger.info("Retour LLM récupéré.")
            content = getattr(response, "content", None)
            if not content:
                logger.warning("Pas de contenu retourné par le LLM ou réponse vide.")
                return "Le LLM n'a rien répondu (réponse vide, sans contenu)"
            return content
        except Exception as e:
            logger.error(f"Erreur lors de la génération de la réponse LLM : {e}")
            return "Une erreur s'est produite lors de la génération de la réponse."


In [37]:
pipeline = RAGPipelineAzure(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Le modèle d'embedding HuggingFace utilisé à l'indexation
    api_key=config["openai_key"],                         # Clé API Azure OpenAI sortie du terraform
    api_base=config["openai_endpoint"],                   # Endpoint Azure OpenAI sorti du terraform
    api_version="2024-02-15-preview",                     # Version API Azure OpenAI (à ajouter dans ta classe, cf plus bas)
    deployment_name=config["chat_deployment"],            # Nom du déploiement Azure OpenAI
    config=config,
    index_name="documents"
)

### Tests simple pour vérifier le résultat

In [38]:
# Réponse
print(pipeline.answer("En quelles couleurs est disponible le modèle urbain classic ?"))

ERROR:__main__:Erreur lors de la génération de la réponse LLM : No module named 'langchain_core.memory'


Une erreur s'est produite lors de la génération de la réponse.


In [ ]:
print(pipeline.answer("Quel est le poids du Urbain-Confort ?"))

In [ ]:
print(pipeline.answer("Que faire si la chaîne du vélo Pro-Livraison saute ?"))

### Test pour une information absente des manuels

In [ ]:
print(pipeline.answer("Quel est le diamètre des roues du vélo Pro-Livraison ?"))